## Quantitaive and qualitative data inspection

In [ ]:
# libraries 
import numpy as np
import h5py
import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.animation as animation
from IPython.display import display
import os
import sys
sys.path.append('../../src')

from utils.evaluate_utils import get_indices
from prepare_data.kspace_sampling import adjust_image_size_centered

### Qualitative analysis

In [ ]:
# h5_files = ['../../data/CARDIAC/M5_2mm_step2_static_dynamic.h5','../../data/CARDIAC/M6_2mm_step2_static_dynamic.h5', ]
h5_files = ['../../results/kspacesampling/FromAlex/result_cs.h5','../../results/kspacesampling/res_cs16_9coils_large.h5', ]
file_label = ['result_cs',]

# columns to load from data
# colnames = ['u', 'v', 'w',  'mask'] #
colnames = ['res_vel']

# load data
data = {}
for h5_file, label in zip(h5_files, file_label):
    with h5py.File(h5_file, 'r') as f:
        for vel in colnames:
            data[f'{label}_{vel}'] = np.array(f[vel]).squeeze()

            if data[f'{label}_{vel}'].dtype == np.complex64:
                data[f'{label}_{vel}_abs'] = np.abs(data[f'{label}_{vel}'])
                data[f'{label}_{vel}_ang'] = np.angle(data[f'{label}_{vel}'])


if 'abs' in data.keys() and 'ang' in data.keys():
    colnames = [f'{col}_abs' for col in colnames] + [f'{col}_ang' for col in colnames]

print(colnames)

In [ ]:

T, X, Y, Z= data[f'{file_label[0]}_{colnames[0]}'].shape

print(T, X, Y, Z)
@widgets.interact(frame=(0, T-1), x = (0, X-1), vel=colnames, axis=[0, 1, 2])
def f(frame=5, x = 10, vel = f'{file_label[0]}_{colnames[0]}', axis = 0):
    plt.figure(figsize=(7,7))
    N = 2
    idxs = get_indices(frame,axis, x)


    plt.subplot(1, N, 1)
    plt.imshow(np.abs(data[f'{label}_{vel}'][idxs]))#,  vmin = min_v[vel[0]], vmax = max_v[vel[0]], cmap='viridis')
    plt.title('abs')
    plt.axis('off')

    plt.subplot(1, N, 2)
    plt.imshow(np.angle(data[f'{label}_{vel}'][idxs]))#,  vmin = min_v[vel[0]], vmax = max_v[vel[0]], cmap='viridis')
    plt.title('angle')
    plt.axis('off')
    

In [ ]:
data_dir = '/mnt/c/Users/piacal/Code/SuperResolution4DFlowMRI/Temporal4DFlowNet/data'
path_datamodel = f'{data_dir}/CARDIAC/M1_2mm_step2_static_dynamic.h5'

# load data
with h5py.File(path_datamodel, mode = 'r') as h5:
    vel_u = np.array(h5['u'])[::2, :, :, :]
    venc_u = np.max(np.array(h5['u_max']))
    mask = np.array(h5['mask']).squeeze()[::2, :, :, :]

max_v = np.max(vel_u)
min_v = np.min(vel_u)


#adjust image size
data_cs = data[f'{file_label[0]}_res_vel']
data_cs = adjust_image_size_centered(data_cs, vel_u.shape)
data_cs *= mask

print(data_cs.shape, data_cs.min(), data_cs.max(),  vel_u.shape)

@widgets.interact(frame=(0, data_cs.shape[0]-1), x = (0, data_cs.shape[1]-1), axis=[0, 1, 2])
def f(frame=5, x = 10,  axis = 0):
    plt.figure(figsize=(7,7))
    N = 2
    idxs = get_indices(frame,axis, x)


    plt.subplot(1, N, 1)
    plt.imshow(data_cs[idxs])#,  vmin = min_v, vmax = max_v, cmap='viridis')
    plt.title('compresses sensing recon')
    plt.axis('off')

    plt.subplot(1, N, 2)
    plt.imshow(vel_u[idxs],  vmin = min_v, vmax = max_v, cmap='viridis')
    plt.title('original')
    plt.axis('off')


In [ ]:
#TODO: add vmin and vmax to the plot
# make use to also choose the colname (data) such that magnitude and velocity can be plotted next to each other

# show interactive plot

T, X, Y, Z= data[f'{file_label[0]}_{colnames[0]}'].shape

print(T, X, Y, Z)
@widgets.interact(frame=(0, T-1), x = (0, X-1), vel=colnames, axis=[0, 1, 2])
def f(frame=5, x = 10, vel = f'{file_label[0]}_{colnames[0]}', axis = 0):
    plt.figure(figsize=(7,7))
    N = len(h5_files)
    idxs = get_indices(frame,axis, x)

    for i, t_name in enumerate(file_label):

        plt.subplot(1, N, i+1)
        plt.imshow(data[f'{label}_{vel}'][idxs])#,  vmin = min_v[vel[0]], vmax = max_v[vel[0]], cmap='viridis')
        plt.title(t_name)
        plt.axis('off')
    

In [ ]:
for vel in colnames:
    print(np.linalg.norm(data[f'{file_label[0]}_{vel}'][:, :, :, 1:] - data[f'{file_label[1]}_{vel}']))

plt.plot(np.mean(data[f'{file_label[0]}_u'], axis=(1, 2, 3)), label='Model5')
plt.plot(np.mean(data[f'{file_label[1]}_u'], axis=(1, 2, 3)), label='Model6')


In [ ]:

def bbox2_3D(img):
    ''' code copied from https://stackoverflow.com/questions/31400769/bounding-box-of-numpy-array'''

    r = np.any(img, axis=(1, 2))
    c = np.any(img, axis=(0, 2))
    z = np.any(img, axis=(0, 1))

    rmin, rmax = np.where(r)[0][[0, -1]]
    cmin, cmax = np.where(c)[0][[0, -1]]
    zmin, zmax = np.where(z)[0][[0, -1]]

    return rmin, rmax, cmin, cmax, zmin, zmax

rmin, rmax, cmin, cmax, zmin, zmax = bbox2_3D(data[f'{file_label[0]}_mask'])

print('mask shape', data[f'{file_label[0]}_mask'].shape[1:])
print(rmin, rmax, cmin, cmax, zmin, zmax)

bbox = np.zeros(data[f'{file_label[0]}_mask'].shape[1:])
bbox[cmin:cmax,zmin:zmax,rmin:rmax] = 1


In [ ]:
from mpl_toolkits.mplot3d import Axes3D
mask = data[f'{file_label[0]}_mask'].squeeze()[0]

x, y, z = np.where(mask == 1)
x1, y1, z1 = np.where(data[f'{file_label[0]}_u'][0] != 0)
x2, y2, z2 = np.where(bbox == 1)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z)
ax.scatter(x1, y1, z1)
ax.scatter(x2, y2, z2, alpha = 0.01, c ='b')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim(0, X)
ax.set_ylim(0, Y)
ax.set_zlim(0, Z)
plt.show()

Create gif Animation of data

In [ ]:
min_v = None
max_v = None

eval_dir = '../../results/interpolation/plots'
if not os.path.exists(eval_dir):
    os.makedirs(eval_dir)

def create_temporal_comparison_gif_single(idx, data, title):
    N_frames = data.shape[0]
    fig = plt.figure(frameon=False)
    im1 = plt.imshow(data[0, idx, :, :],interpolation='none', vmin=min_v, vmax=max_v)
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()

    #initialization function: plot the background of each frame
    def init():
        im1.set_data(np.random.random((5,5)))
        return [im1]

    # animation function.  This is called sequentially
    def animate(i):
        im1.set_array(data[i, idx, :, :])
        return [im1]

    anim = animation.FuncAnimation(fig,animate, init_func=init, frames = N_frames,interval = 100, ) # in ms)


    anim.save(f'{eval_dir}/Animate_{title}_single.gif', fps=10)

t, x, y, z = data[f'{file_label[0]}_u'].shape
create_temporal_comparison_gif_single(x//2, data[f'{file_label[0]}_v'], file_label[0])